In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

In [3]:
input_text = au.read_txt_file_lines()
n_rows = len(input_text)

In [6]:
for l in input_text:
    if l == '':
        print(input_text.index(l))
        ind = input_text.index(l) 

input_ops = input_text[:ind] 
input_data = input_text[ind+1:]

506


In [15]:
## store parts as named tuples
part = collections.namedtuple('part', 'x m a s')
parts = []
for l in input_data:
    tmp = l.lstrip('{').rstrip('}').split(',')
    assert len(tmp) == 4 
    assert tmp[0].startswith('x=') and tmp[1].startswith('m=') and tmp[2].startswith('a=') and tmp[3].startswith('s=')
    tmp = [int(x.split('=')[1]) for x in tmp]
    parts.append(part(*tmp))

In [74]:
def generic_op(str_op):
    '''given a str (like "a<123:R" or "A"), return a function that takes a part and returns True/False + outcome'''
    if len(str_op) == 1:
        if str_op == 'R':
            return (lambda x: True, 'R')
        elif str_op == 'A':
            return (lambda x: True, 'A')
        else:
            assert False, 'unknown op'
    elif len(str_op) > 1 and ('<' in str_op or '>' in str_op):
        tmp = str_op.split(':')
        assert len(tmp) == 2, f'{str_op}'
        cond = tmp[0]
        result = tmp[1]
        assert cond[1] in ['<', '>'], f'{cond} not < >'
        operation = cond[1]
        cond = cond.split(operation)
        assert len(cond) == 2
        component = cond[0]
        value = int(cond[1])
        assert len(component) == 1 and component in ['x', 'm', 'a', 's']
        if operation == '<':
            return lambda x: getattr(x, component) < value, result
        elif operation == '>':
            return lambda x: getattr(x, component) > value, result
        else:
            assert False, 'unknown operation'
    else:   ## box name 
        return (lambda x: True, str_op)

def evaluate_op(str_op, part):
    '''given a str_op and part, evaluate op and return result if True'''
    op_fun, result_if_true = generic_op(str_op)
    # print(op_fun(part), result_if_true)
    if op_fun(part):
        return result_if_true 
    else:
        return None 
    
def evaluate_box(box_name, part):
    '''loop through ops in box and evaluate until result is found. recursively call other boxes if needed. assert false if end is reached without result'''
    ops = dict_boxes[box_name]
    for op in ops:
        res = evaluate_op(op, part)
        # print(op, res)
        if res is not None:
            if res in ['A', 'R']:
                return res 
            else: 
                return evaluate_box(res, part)    
    assert False, 'no result found'
    
## create dict with boxes with lsit of str_ops
dict_boxes = {} 
for l in input_ops:
    tmp = l.split('{')
    assert len(tmp) == 2, f'{l}'
    name_box = tmp[0]
    all_ops = tmp[1].rstrip('}').split(',')
    assert name_box not in dict_boxes
    dict_boxes[name_box] = [x for x in all_ops]


In [75]:
total_accepted = 0
for part_curr in parts:
    res = evaluate_box('in', part_curr)
    if res == 'A':
        total_accepted += (part_curr.x + part_curr.m + part_curr.a + part_curr.s) 

print(total_accepted)

353046


## part 2: 

excercise: of all possible parts with x,m,a,s in (0, 4000), how many combinations are possible?

bit like day 5, which I solved very *hackily* at the time.. 